In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

2024-07-13 20:25:53.273698: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Global Params

In [3]:
model_name = 'simulation-hawkes'
n_sims = 10
inputs_path = os.path.join(os.getcwd(), 'data', 'inputs')
outputs_path = os.path.join(os.getcwd(), 'data', 'outputs')

## Load Simulation

In [4]:
ric = "fake"
dataPath = os.path.join(outputs_path, model_name)
dictBinnedData = {}
for d in range(n_sims):
    dictBinnedData[d] = []
dates = list(dictBinnedData.keys())

dfs = []
for i in dates:
    try:
        read_path = os.path.join(dataPath, ric + "_" + str(i) + "_12D.csv")
        df = pd.read_csv(read_path)

        # drop columns
        df.drop(['Unnamed: 0'], axis=1, inplace=True)

        # change columns order
        df = df[["Date", "Time", "event"] + df.drop(["Date", "Time", "event"], axis=1).columns.to_list()]

        dfs.append(df)
    except:
        print(f"No data for {ric} on {i}")
df = pd.concat(dfs)

df.tail()

,Date,Time,event,Ask Price 1,Bid Price 1,Ask Price 2,Bid Price 2,BidDiff,AskDiff,BidDiff2,AskDiff2
347,9,98.942260,co_deep_Ask,45.0,44.99,45.02,44.98,0.0,0.0,0.0,0.0
348,9,98.942339,co_deep_Ask,45.0,44.99,45.02,44.98,0.0,0.0,0.0,0.0
349,9,98.942505,co_deep_Ask,45.0,44.99,45.02,44.98,0.0,0.0,0.0,0.0
350,9,98.942836,co_deep_Ask,45.0,44.99,45.02,44.98,0.0,0.0,0.0,0.0
351,9,106.865217,co_deep_Bid,45.0,44.99,45.02,44.98,0.0,0.0,0.0,0.0


In [5]:
df["event"].unique()

array(['co_top_Ask', 'mo_Bid', 'lo_top_Bid', 'lo_inspread_Bid',
       'lo_inspread_Ask', 'co_deep_Bid', 'mo_Ask', 'lo_deep_Bid',
       'lo_top_Ask', 'co_top_Bid', 'co_deep_Ask', 'lo_deep_Ask'],
      dtype=object)

## Prepare Data

In [6]:
pivot_df = df.copy()
pivot_df["count"] = 1

pivot_df = pivot_df.pivot_table(index=["Date", "Time"], columns="event", values="count").fillna(0)

X = pivot_df.values

pivot_df.head()

event          co_deep_Ask  co_deep_Bid  co_top_Ask  co_top_Bid  lo_deep_Ask  \
Date Time                                                                      
0    0.151024          0.0          0.0         1.0         0.0          0.0   
     0.159023          0.0          0.0         0.0         0.0          0.0   
     0.159032          0.0          0.0         0.0         0.0          0.0   
     0.159052          0.0          0.0         0.0         0.0          0.0   
     0.159081          0.0          0.0         0.0         0.0          0.0   

event          lo_deep_Bid  lo_inspread_Ask  lo_inspread_Bid  lo_top_Ask  \
Date Time                                                                  
0    0.151024          0.0              0.0              0.0         0.0   
     0.159023          0.0              0.0              0.0         0.0   
     0.159032          0.0              0.0              0.0         0.0   
     0.159052          0.0              0.0              1.0         0.0   
     0.159081          0.0              0.0              1.0         0.0   

event          lo_top_Bid  mo_Ask  mo_Bid  
Date Time                                  
0    0.151024         0.0     0.0     0.0  
     0.159023         0.0     0.0     1.0  
     0.159032         1.0     0.0     0.0  
     0.159052         0.0     0.0     0.0  
     0.159081         0.0     0.0     0.0

In [7]:
X.shape

(3793, 12)

## Log-likelihood for a M-variate Hawkes Process with Exponential Excitation Kernels

### 1) Single observation


#### Intuitive Version

In [8]:
# model init params
np.random.seed(42)
M = X.shape[1]
T = X.shape[0]
ts = np.arange(0, T)

# model estimation params
mu = np.random.uniform(0.1, 1.0, M)
alpha = np.random.uniform(0.01, 0.5, (M, M))
beta = np.random.uniform(0.5, 2.0, (M, M))

# init recursive function
R = np.zeros((M, T, M))

for m in range(M):

    for n in range(M):

        tks = np.nonzero(X[:, m])[0]
        tis = np.nonzero(X[:, n])[0]
        for t in range(len(tks)):
            if t == 0:
                continue
            else:
                tk = tks[t]
                tkm1 = tks[t - 1]

                ti = tis[tis < tk]
                ti = ti[ti > tkm1]

                sum_over_tis_given_n = 0
                for s in range(len(ti)):
                    if s == 0:
                        continue
                    else:
                        sum_over_tis_given_n += np.exp(-beta[m, n] * (ti[s] - ti[s - 1]))

                R[m, t, n] = np.exp(-beta[m, n] * (tk - tkm1)) * R[m, t - 1, n] + sum_over_tis_given_n

log_likelihood = 0
for m in range(M):

     integral_term_over_t = 0
     for n in range(M):
          for t in ts:
               integral_term_over_t += (alpha[m, n] / beta[m, n] * (1 - np.exp(-beta[m, n] * (T - t))))
     integral_term_over_t = -(mu[m] * T) - integral_term_over_t

     integra_term_over_countp = 0
     for t in ts:
          for n in range(M):
               integra_term_over_countp += np.log(mu[n] + (alpha[m, n] * R[m, t, n]))

     log_likelihood += integral_term_over_t + integra_term_over_countp

In [9]:
print(f"Log-Likelihood: {log_likelihood}")

Log-Likelihood: -566307.6579780579


#### Slightly Optimized Version

In [10]:
# Model initialization parameters
np.random.seed(42)
M = X.shape[1]
T = X.shape[0]
ts = np.arange(0, T)

# Model estimation parameters
mu = np.random.uniform(0.1, 1.0, M)
alpha = np.random.uniform(0.01, 0.5, (M, M))
beta = np.random.uniform(0.5, 2.0, (M, M))

# Initialize recursive function
R = np.zeros((M, T, M))

for m in range(M):
    for n in range(M):
        tks = np.nonzero(X[:, m])[0]
        tis = np.nonzero(X[:, n])[0]

        # Vectorized calculation of R
        if len(tks) > 1:
            for t in range(1, len(tks)):
                tk = tks[t]
                tkm1 = tks[t - 1]
                ti = tis[(tis > tkm1) & (tis < tk)]

                sum_over_tis_given_n = 0
                for s in range(len(ti)):
                    if s == 0:
                        continue
                    else:
                        sum_over_tis_given_n += np.exp(-beta[m, n] * (ti[s] - ti[s - 1]))

                R[m, t, n] = np.exp(-beta[m, n] * (tk - tkm1)) * R[m, t - 1, n] + sum_over_tis_given_n

log_likelihood = 0
for m in range(M):
    integral_term_over_t = 0
    for n in range(M):
        for t in ts:
            integral_term_over_t += (alpha[m, n] / beta[m, n] * (1 - np.exp(-beta[m, n] * (T - t))))
    integral_term_over_t = -(mu[m] * T) - integral_term_over_t

    integra_term_over_countp = 0
    for t in ts:
        for n in range(M):
            integra_term_over_countp += np.log(mu[m] + (alpha[m, n] * R[m, t, n]))

    log_likelihood += integral_term_over_t + integra_term_over_countp

In [11]:
print(f"Log-Likelihood: {log_likelihood}")

Log-Likelihood: -566112.5257094807


#### Slightly Optimized Version - Vectorize Inner Loop

In [12]:
# Model initialization parameters
np.random.seed(42)
M = X.shape[1]
T = X.shape[0]
ts = np.arange(0, T)

# Model estimation parameters
mu = np.random.uniform(0.1, 1.0, M)
alpha = np.random.uniform(0.01, 0.5, (M, M))
beta = np.random.uniform(0.5, 2.0, (M, M))

# Initialize recursive function
R = np.zeros((M, T, M))

for m in range(M):
    for n in range(M):
        tks = np.nonzero(X[:, m])[0]
        tis = np.nonzero(X[:, n])[0]

        # Vectorized calculation of R
        if len(tks) > 1:
            for t in range(1, len(tks)):
                tk = tks[t]
                tkm1 = tks[t - 1]
                ti = tis[(tis > tkm1) & (tis < tk)]

                if len(ti) > 1:
                    # Vectorized sum of exponentials
                    ti_diff = np.diff(ti)
                    exp_terms = np.exp(-beta[m, n] * ti_diff)
                    sum_over_tis_given_n = np.sum(exp_terms)

                    R[m, t, n] = np.exp(-beta[m, n] * (tk - tkm1)) * R[m, t - 1, n] + sum_over_tis_given_n
                elif len(ti) == 1:
                    R[m, t, n] = np.exp(-beta[m, n] * (tk - tkm1)) * R[m, t - 1, n] + np.exp(-beta[m, n] * (ti[0] - tkm1))

log_likelihood = 0
for m in range(M):
    integral_term_over_t = 0
    for n in range(M):
        for t in ts:
            integral_term_over_t += (alpha[m, n] / beta[m, n] * (1 - np.exp(-beta[m, n] * (T - t))))
    integral_term_over_t = -(mu[m] * T) - integral_term_over_t

    integra_term_over_countp = 0
    for t in ts:
        for n in range(M):
            integra_term_over_countp += np.log(mu[m] + (alpha[m, n] * R[m, t, n]))

    log_likelihood += integral_term_over_t + integra_term_over_countp

In [13]:
print(f"Log-Likelihood: {log_likelihood}")

Log-Likelihood: -566526.5973734233


### 2) Full dataset

In [14]:
def exponential_excitation_log_likelihoodI(X, mu, alpha, beta):
    """
    Computes the log-likelihood of an M-variate Hawkes process using TensorFlow.

    Parameters:
    X (np.array): Event matrix with shape (T, M), where T is the number of time steps and M is the number of dimensions.
    mu (tf.Variable): Base intensity vector with shape (M,)
    alpha (tf.Variable): Excitation matrix with shape (M, M)
    beta (tf.Variable): Decay parameter matrix with shape (M, M)

    Returns:
    float: Negative log-likelihood of the Hawkes process
    """

    M = X.shape[1]
    T = X.shape[0]
    ts = np.arange(0, T)

    # Initialize recursive function
    R = tf.zeros((M, T, M))

    # Compute R recursively
    for m in range(M):
        for n in range(M):
            tks = np.nonzero(X[:, m])[0]
            tis = np.nonzero(X[:, n])[0]

            if len(tks) > 1:
                for t in range(1, len(tks)):
                    tk = tks[t]
                    tkm1 = tks[t - 1]
                    ti = tis[(tis > tkm1) & (tis < tk)]

                    sum_over_tis_given_n = 0
                    for s in range(len(ti)):
                        if s == 0:
                            continue
                        else:
                            sum_over_tis_given_n += tf.exp(-beta[m, n] * (ti[s] - ti[s - 1]))

                    R = tf.tensor_scatter_nd_update(R, [[m, t, n]], [tf.exp(-beta[m, n] * (tk - tkm1)) * R[m, t - 1, n] + sum_over_tis_given_n])

    log_likelihood = 0
    for m in range(M):
        integral_term_over_t = 0
        for n in range(M):
            for t in ts:
                integral_term_over_t += (alpha[m, n] / beta[m, n] * (1 - tf.exp(-beta[m, n] * (T - t))))
        integral_term_over_t = -(mu[m] * T) - integral_term_over_t

        integra_term_over_countp = 0
        for t in ts:
            for n in range(M):
                integra_term_over_countp += tf.math.log(mu[m] + (alpha[m, n] * R[m, t, n]))

        log_likelihood += integral_term_over_t + integra_term_over_countp

    return -log_likelihood  # Return negative log-likelihood for minimization

In [17]:
def exponential_excitation_log_likelihoodII(X, mu, alpha, beta):
    """
    Computes the log-likelihood of an M-variate Hawkes process using TensorFlow.

    Parameters:
    X (np.array): Event matrix with shape (T, M), where T is the number of time steps and M is the number of dimensions.
    mu (tf.Variable): Base intensity vector with shape (M,)
    alpha (tf.Variable): Excitation matrix with shape (M, M)
    beta (tf.Variable): Decay parameter matrix with shape (M, M)

    Returns:
    float: Negative log-likelihood of the Hawkes process
    """

    M = X.shape[1]
    T = X.shape[0]
    ts = np.arange(0, T)

    # Initialize recursive function
    R = tf.zeros((M, T, M))

    # Compute R recursively
    for m in range(M):
        for n in range(M):
            tks = np.nonzero(X[:, m])[0]
            tis = np.nonzero(X[:, n])[0]

            if len(tks) > 1:
                for t in range(1, len(tks)):
                    tk = tks[t]
                    tkm1 = tks[t - 1]
                    ti = tis[(tis > tkm1) & (tis < tk)]

                    if len(ti) > 1:
                        # Compute differences using slicing
                        ti_diff = ti[1:] - ti[:-1]
                        exp_terms = tf.exp(-beta[m, n] * tf.cast(ti_diff, dtype=tf.float32))
                        sum_over_tis_given_n = tf.reduce_sum(exp_terms)

                        R = tf.tensor_scatter_nd_update(R, [[m, t, n]], [tf.exp(-beta[m, n] * tf.cast((tk - tkm1), dtype=tf.float32)) * R[m, t - 1, n] + sum_over_tis_given_n])
                    elif len(ti) == 1:
                        sum_over_tis_given_n = tf.exp(-beta[m, n] * tf.cast((ti[0] - tkm1), dtype=tf.float32))
                        R = tf.tensor_scatter_nd_update(R, [[m, t, n]], [tf.exp(-beta[m, n] * tf.cast((tk - tkm1), dtype=tf.float32)) * R[m, t - 1, n] + sum_over_tis_given_n])

    log_likelihood = 0
    for m in range(M):
        integral_term_over_t = 0
        for n in range(M):
            for t in ts:
                integral_term_over_t += (alpha[m, n] / beta[m, n] * (1 - tf.exp(-beta[m, n] * (T - t))))
        integral_term_over_t = -(mu[m] * T) - integral_term_over_t

        integra_term_over_countp = 0
        for t in ts:
            for n in range(M):
                integra_term_over_countp += tf.math.log(mu[m] + (alpha[m, n] * R[m, t, n]))

        log_likelihood += integral_term_over_t + integra_term_over_countp

    return -log_likelihood  # Return negative log-likelihood for minimization

In [17]:
# Model initialization parameters
X = pivot_df.values
M = X.shape[1]

# Init Model estimation parameters
mu = tf.Variable(np.random.uniform(0.1, 1.0, M), dtype=tf.float32)
alpha = tf.Variable(np.random.uniform(0.01, 0.5, (M, M)), dtype=tf.float32)
beta = tf.Variable(np.random.uniform(0.5, 2.0, (M, M)), dtype=tf.float32)

# Define optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# Optimization step
@tf.function
def train_step():
    with tf.GradientTape() as tape:
        loss = exponential_excitation_log_likelihoodI(X, mu, alpha, beta)
    gradients = tape.gradient(loss, [mu, alpha, beta])
    optimizer.apply_gradients(zip(gradients, [mu, alpha, beta]))
    return loss

# Perform optimization
for epoch in range(5):
    loss = train_step()
    if epoch % 1 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

2024-07-13 20:17:06.967150: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
# Model initialization parameters
X = pivot_df.values
M = X.shape[1]

# Init Model estimation parameters
mu = tf.Variable(np.random.uniform(0.1, 1.0, M), dtype=tf.float32)
alpha = tf.Variable(np.random.uniform(0.01, 0.5, (M, M)), dtype=tf.float32)
beta = tf.Variable(np.random.uniform(0.5, 2.0, (M, M)), dtype=tf.float32)

# Define optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# Optimization step
@tf.function
def train_step():
    with tf.GradientTape() as tape:
        loss = exponential_excitation_log_likelihoodII(X, mu, alpha, beta)
    gradients = tape.gradient(loss, [mu, alpha, beta])
    optimizer.apply_gradients(zip(gradients, [mu, alpha, beta]))
    return loss

# Perform optimization
for epoch in range(5):
    loss = train_step()
    if epoch % 1 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")